# NY Motor Vehicle Collisions – Exploratory Data Analysis  
In this notebook, we analyze New York’s Vehicular crash data available at https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95


The data contains information from all police reported motor vehicle collisions in NYC. The information for this dataset is collated from the police report, called MV104-AN, which is required to be filled out for collisions where someone is injured or killed, or where there is at least $1000 worth of damage.

Data is available from 2012-07-01 onwards, however for this analysis, we will limit ourselves to the period up to 2023-08-15, which is when the data was downloaded.

We find that this data contains over 2 million observations, which allows us to explore several aspects of vehicle crashes in NY's boroughs.

The data dictionary for the data is also available at the URL above.

First, we perform unstructured exploration of the data, and then try to answer the following questions:

- We look for which borough has had the maximum number of crashes reported since 2012.

- We relate the number of crashes to to the borough's population to find out which borough has the maximum number of crashes for every 100,000 people. Even though the data does not have this information, we can combine the crash data with the population estimates for the boroughs also available from the City of New York's website (https://data.cityofnewyork.us/City-Government/New-York-City-Population-by-Borough-1950-2040/xywu-7bv9)

| Borough | Population |
| --- | --- |
| Bronx | 1446788 |
| Brooklyn | 2648452 |
| Manhattan | 1638281 |
| Queens | 2330295 |
| Staten Island | 487155 |  

  
- We look for the leading cause of crashes

- We also look for the top-3 causes of crashes, and try to calculate what proportion of all crashes are caused by these top-3 causes.

- We then look to some of the more serious implications of crashes by examining how many accidents involved at least one fatality.

- We then compute, on average, out of every 1000 accidents, how many have resulted in at least one person dead.

- We also look for missing data and try to compute the proportion of accidents in the data that do not have a Borough code.

- The fields 'VEHICLE TYPE CODE 1' and 'VEHICLE TYPE CODE 2' represent the first two vehicles involved in the accident.  We look for which combinations of vehicles have the most number of accidents.


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt
import phik
from IPython.display import Markdown as md

In [2]:
df = pd.read_pickle("shared/Motor_Vehicle_Collisions_-_Crashes.pkl")
df


,CRASH DATE_CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2021-09-11 02:39:00,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,2.0,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,2022-03-26 11:45:00,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,2022-06-29 06:55:00,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
3,2021-09-11 09:35:00,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,0.0,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
4,2021-12-14 08:13:00,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018240,2023-07-03 18:05:00,NaN,NaN,40.866806,-73.931010,"(40.866806, -73.93101)",RIVERSIDE DRIVE,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4648110,Sedan,Sedan,NaN,NaN,NaN
2018241,2023-07-22 21:39:00,BRONX,10457.0,40.844177,-73.902920,"(40.844177, -73.90292)",EAST 174 STREET,WEBSTER AVENUE,NaN,1.0,...,NaN,NaN,NaN,NaN,4648117,Sedan,NaN,NaN,NaN,NaN
2018242,2023-07-02 17:55:00,MANHATTAN,10006.0,40.711033,-74.014540,"(40.711033, -74.01454)",WEST STREET,LIBERTY STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4648366,Taxi,NaN,NaN,NaN,NaN
2018243,2023-07-22 13:15:00,QUEENS,11433.0,40.691580,-73.793190,"(40.69158, -73.79319)",110 AVENUE,157 STREET,NaN,1.0,...,Driver Inattention/Distraction,NaN,NaN,NaN,4648129,Station Wagon/Sport Utility Vehicle,E-Bike,NaN,NaN,NaN


***
### Show some exploratory data analysis here
***

## Borough with the maximum number of crashes reported since 2012

In [6]:
#list total number of crashes within different borough
df.BOROUGH.value_counts().dropna().sort_values(ascending=False)

BOROUGH
BROOKLYN         441026
QUEENS           372457
MANHATTAN        313266
BRONX            205345
STATEN ISLAND     58297
Name: count, dtype: int64

In [7]:
#find the top 1
df.BOROUGH.value_counts().dropna().sort_values(ascending=False).head(1)

BOROUGH
BROOKLYN    441026
Name: count, dtype: int64

## Borough with the maximum number of crashes adjusted for population

In [8]:
#Insert population dataframe
pop = pd.DataFrame(data = {'BOROUGH':['BRONX', 'BROOKLYN', 'MANHATTAN', 'QUEENS', 'STATEN ISLAND'],
             'POPULATION': [1446788, 2648452, 1638281, 2330295, 487155]})  

pop

,BOROUGH,POPULATION
0,BRONX,1446788
1,BROOKLYN,2648452
2,MANHATTAN,1638281
3,QUEENS,2330295
4,STATEN ISLAND,487155


In [9]:
#Give this list a name 'crash'
crash=df.BOROUGH.value_counts().dropna()

In [10]:
#Combine two datasets, and named as 'total'
total=pop.merge(crash, how = 'left' , 
         left_on = 'BOROUGH', right_on = 'BOROUGH')
total

,BOROUGH,POPULATION,count
0,BRONX,1446788,205345
1,BROOKLYN,2648452,441026
2,MANHATTAN,1638281,313266
3,QUEENS,2330295,372457
4,STATEN ISLAND,487155,58297


In [11]:
#Apply calculation
total['Adjusted_crash'] = total['count']/total['POPULATION']*100000
total

,BOROUGH,POPULATION,count,Adjusted_crash
0,BRONX,1446788,205345,14193.164444
1,BROOKLYN,2648452,441026,16652.217975
2,MANHATTAN,1638281,313266,19121.628097
3,QUEENS,2330295,372457,15983.255339
4,STATEN ISLAND,487155,58297,11966.827806


## Analyzing the leading cause of crashes

In [12]:
#Extract the column 'CONTRIBUTING FACTOR VEHICLE 1'
fac1=df.loc[:,'CONTRIBUTING FACTOR VEHICLE 1']
fac1

0            Aggressive Driving/Road Rage
1                       Pavement Slippery
2                   Following Too Closely
3                             Unspecified
4                                     NaN
                        ...              
2018240                Turning Improperly
2018241                       Unspecified
2018242    Driver Inattention/Distraction
2018243    Driver Inattention/Distraction
2018244                       Unspecified
Name: CONTRIBUTING FACTOR VEHICLE 1, Length: 2018245, dtype: object

In [13]:
#Replace 'Unspecified' with null value
fac1.replace('Unspecified',np.nan,inplace=True)
fac1

0            Aggressive Driving/Road Rage
1                       Pavement Slippery
2                   Following Too Closely
3                                     NaN
4                                     NaN
                        ...              
2018240                Turning Improperly
2018241                               NaN
2018242    Driver Inattention/Distraction
2018243    Driver Inattention/Distraction
2018244                               NaN
Name: CONTRIBUTING FACTOR VEHICLE 1, Length: 2018245, dtype: object

In [14]:
#Drop null value and calculate the percentage of different reasons 
fac1.value_counts(dropna = True, normalize = True).sort_values(ascending=False)

CONTRIBUTING FACTOR VEHICLE 1
Driver Inattention/Distraction                           0.304180
Failure to Yield Right-of-Way                            0.090335
Following Too Closely                                    0.081466
Backing Unsafely                                         0.056886
Other Vehicular                                          0.047521
Passing or Lane Usage Improper                           0.042031
Turning Improperly                                       0.037833
Passing Too Closely                                      0.037788
Fatigued/Drowsy                                          0.035889
Unsafe Lane Changing                                     0.030103
Traffic Control Disregarded                              0.026691
Driver Inexperience                                      0.023707
Unsafe Speed                                             0.020403
Alcohol Involvement                                      0.016416
Lost Consciousness                            

## Top 3 causes of crashes 

In [15]:
#Find the first three causes from the previous question and add them up to find the total
fac1.value_counts(dropna = True, normalize = True).sort_values(ascending=False).head(3).sum()

0.47598056643578757

## Analyzing fatalities

In [16]:
#Extract columns 'CRASH DATE_CRASH TIME' and 'NUMBER OF PERSONS KILLED', named as 'fata'
fata=df.loc[:,['CRASH DATE_CRASH TIME','NUMBER OF PERSONS KILLED']]
fata

,CRASH DATE_CRASH TIME,NUMBER OF PERSONS KILLED
0,2021-09-11 02:39:00,0.0
1,2022-03-26 11:45:00,0.0
2,2022-06-29 06:55:00,0.0
3,2021-09-11 09:35:00,0.0
4,2021-12-14 08:13:00,0.0
...,...,...
2018240,2023-07-03 18:05:00,0.0
2018241,2023-07-22 21:39:00,0.0
2018242,2023-07-02 17:55:00,0.0
2018243,2023-07-22 13:15:00,0.0


In [17]:
#Replace 0 with NaN, since 0 is not useful here
fata.replace(0,np.nan,inplace=True)
fata

,CRASH DATE_CRASH TIME,NUMBER OF PERSONS KILLED
0,2021-09-11 02:39:00,NaN
1,2022-03-26 11:45:00,NaN
2,2022-06-29 06:55:00,NaN
3,2021-09-11 09:35:00,NaN
4,2021-12-14 08:13:00,NaN
...,...,...
2018240,2023-07-03 18:05:00,NaN
2018241,2023-07-22 21:39:00,NaN
2018242,2023-07-02 17:55:00,NaN
2018243,2023-07-22 13:15:00,NaN


In [18]:
#Drop all null value, and calculate the total number of remaining
fata.dropna().count()

CRASH DATE_CRASH TIME       2804
NUMBER OF PERSONS KILLED    2804
dtype: int64

## Likelihood of fatal accidents
#### On average, out of every 1000 accidents, how many have resulted in at least one person dead?

In [19]:
#calculate the total accidents
total_number_of_accidents=len(df)

In [20]:
#Give the count of accidents with 1 or more deaths
fatal_accidents=2804

In [21]:
#Apply the formula
Likelihood_of_fatal_accidents = fatal_accidents/total_number_of_accidents*1000
Likelihood_of_fatal_accidents

1.3893258747079764

## Missing data
#### What proportion of accidents in the data do not have a Borough code?

In [22]:
#Extract columns 'BOROUGH'
borough=df[['BOROUGH']]
borough

,BOROUGH
0,NaN
1,NaN
2,NaN
3,BROOKLYN
4,BROOKLYN
...,...
2018240,NaN
2018241,BRONX
2018242,MANHATTAN
2018243,QUEENS


In [23]:
#Count null value
count=borough.isna().sum()

In [24]:
#Apply the formula
proportion=(count/len(df))*100
proportion

BOROUGH    31.108909
dtype: float64

## Which combinations of vehicles have the most number of accidents

In [25]:
#Extract columns CRASH DATE_CRASH TIME, VEHICLE TYPE CODE 1, and VEHICLE TYPE CODE 2
veh=df.loc[:,['CRASH DATE_CRASH TIME','VEHICLE TYPE CODE 1','VEHICLE TYPE CODE 2']]
veh

,CRASH DATE_CRASH TIME,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2
0,2021-09-11 02:39:00,Sedan,Sedan
1,2022-03-26 11:45:00,Sedan,NaN
2,2022-06-29 06:55:00,Sedan,Pick-up Truck
3,2021-09-11 09:35:00,Sedan,NaN
4,2021-12-14 08:13:00,NaN,NaN
...,...,...,...
2018240,2023-07-03 18:05:00,Sedan,Sedan
2018241,2023-07-22 21:39:00,Sedan,NaN
2018242,2023-07-02 17:55:00,Taxi,NaN
2018243,2023-07-22 13:15:00,Station Wagon/Sport Utility Vehicle,E-Bike


In [26]:
#Create a combination column
veh['Comb']=veh['VEHICLE TYPE CODE 1']+'+'+veh['VEHICLE TYPE CODE 2']
veh

,CRASH DATE_CRASH TIME,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,Comb
0,2021-09-11 02:39:00,Sedan,Sedan,Sedan+Sedan
1,2022-03-26 11:45:00,Sedan,NaN,NaN
2,2022-06-29 06:55:00,Sedan,Pick-up Truck,Sedan+Pick-up Truck
3,2021-09-11 09:35:00,Sedan,NaN,NaN
4,2021-12-14 08:13:00,NaN,NaN,NaN
...,...,...,...,...
2018240,2023-07-03 18:05:00,Sedan,Sedan,Sedan+Sedan
2018241,2023-07-22 21:39:00,Sedan,NaN,NaN
2018242,2023-07-02 17:55:00,Taxi,NaN,NaN
2018243,2023-07-22 13:15:00,Station Wagon/Sport Utility Vehicle,E-Bike,Station Wagon/Sport Utility Vehicle+E-Bike


In [27]:
#Count and find the top 1
veh.Comb.value_counts().dropna().sort_values(ascending=False).head(1)

Comb
Sedan+Sedan    197944
Name: count, dtype: int64

***  
## Finished